This research is about housing market of Canada. The reason why I come up this idea is that some many friends told me that the housing market is increasing a lot this year. Hence, I decide to look into the housing market and hopefully I can get some insight from the data.

Before analyzing data, I have to have some data on hand. So I feel like that I can try some web scraping from website which is posting housing information. The first website I tried is a website called 'Point2homes'. As a resident who have stayed in Nova Scotia for six years, I think that I should start this province first.

Here we go, let's try to get some data from it.

In [4]:
# Import requests
import requests
from bs4 import BeautifulSoup
import pandas as pd

We start with Nova Scotia.

In [2]:
url = 'https://www.point2homes.com/CA/Real-Estate-Maps.html?location=Nova+Scotia&PropertyType=House&search_mode=location&page=1&SelectedView=map_view&LocationGeoId=205403&viewport=-67.21871554643741%2C42.95585923041173%2C-58.88009249956241%2C47.442429466642906&zoom=7&location_changed=&ajax=1'

In [3]:
requests.get(url).status_code

200

In [4]:
data=requests.get(url).text

In [5]:
soup = BeautifulSoup(data, 'html.parser')

In [6]:
tables= soup.find_all("div", class_="item-details")

In [7]:
tables[0]

<div class="item-details">
<div class="item-header-cnt">
<div class="characteristics-cnt">
<ul>
<li class="ic-beds" data-label="Beds">
<strong>3</strong> <span class="gray normal-lbl">Beds</span><span class="gray short-lbl">Bds</span>
</li>
<li class="ic-baths" data-label="Bath">
<strong>1</strong> <span class="gray normal-lbl">Bath</span><span class="gray short-lbl">Ba</span>
</li>
<li class="ic-sqft" data-label="Sqft">
<strong>1,630</strong> <span class="gray normal-lbl">Sqft</span><span class="gray short-lbl">Sqft</span>
</li>
<li class="ic-lotsize" data-label="Lot Size">
<strong>3.81 ac</strong> <span class="gray">Lot Size</span>
</li>
<li class="property-type ic-proptype">
            Residential
        </li>
</ul>
</div>
<div class="item-header-left-cnt">
<div class="price" data-price="$225,000 CAD">
<span class="green">
<span>
                $225,000 CAD
            </span>
</span>
<span class="pricedrop-container"> <span class="price-drop" title="24.9K">24.9K</span> </span>
<

In the beginning I found that this web page has 100 information of houses, so I want to check whether I got them all.

len(tables)

Now I want to get all the pages which are 2101 results of housing information. It should be 21 pages of 100 information plus 21 information at the 22nd page.

In [1]:
for i in range(1,23):
    url='https://www.point2homes.com/CA/Real-Estate-Maps.html?location=Nova+Scotia&PropertyType=House&search_mode=location&page=' + str(i) + '&SelectedView=map_view&LocationGeoId=205403&viewport=-67.21871554643741%2C42.95585923041173%2C-58.88009249956241%2C47.442429466642906&zoom=7&location_changed=&ajax=1'
    
print(url)

https://www.point2homes.com/CA/Real-Estate-Maps.html?location=Nova+Scotia&PropertyType=House&search_mode=location&page=22&SelectedView=map_view&LocationGeoId=205403&viewport=-67.21871554643741%2C42.95585923041173%2C-58.88009249956241%2C47.442429466642906&zoom=7&location_changed=&ajax=1


The informaton can be collected are "ic-beds", "ic-baths", "ic-sqft", "ic-lotsize", "property-type ic-proptype", "price" and "item-address"

Let's store them in the data set

In [95]:
data = {}
soups = {}
tables_NS = []
# Create a df to store info
Point2_data = pd.DataFrame(columns=['Beds','Bath','Sqft','Lot_Size','Type','Price','Address'])
for i in range(1,23):
    url='https://www.point2homes.com/CA/Real-Estate-Maps.html?location=Nova+Scotia&PropertyType=House&search_mode=location&page=' + str(i) + '&SelectedView=map_view&LocationGeoId=205403&viewport=-67.21871554643741%2C42.95585923041173%2C-58.88009249956241%2C47.442429466642906&zoom=7&location_changed=&ajax=1'
    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')
    tables_NS.append(soup.find_all("div", class_="item-details"))
    
    # extract information from each table
    for table in (tables_NS[(i-1)]):
        if (len(str(table.find_all("li", class_='ic-beds')).split())<3):
            Nbed = float("NaN")
        else:
            Nbed = int(''.join(filter(str.isdigit, str(table.find_all("li", class_='ic-beds')))))
        
        if (len(str(table.find_all("li", class_='ic-baths')).split())<3):
            NBath = float("NaN")
        else:
            NBath = int(''.join(filter(str.isdigit, str(table.find_all("li", class_='ic-baths')))))
        
        if (len(str(table.find_all("li", class_='ic-sqft')).split())<3):
            NSqft = float("NaN")
        else:
            NSqft = int(''.join(filter(str.isdigit, str(table.find_all("li", class_='ic-sqft')))))
    
        if (len(str(table.find_all("li", class_='ic-lotsize')).split())<4):
            NLot_Size = float("NaN")
        else:
            NLot_Size = str(table.find_all("li", class_='ic-lotsize')).split()[4][-4:]
        
        if (len(str(table.find_all("li", class_='ic-proptype')).split())<3):
            NType = float("NaN")
        else:
            NType = str(table.find_all("li", class_='ic-proptype')).split()[3]


        NPrice = int(table.find("div", class_='price')['data-price'].replace('$','').replace(',','').replace('CAD',''))
        NAddress = table.find("div", class_='item-address')['data-address']
    # append data into the table 
        Point2_data = Point2_data.append({'Beds':Nbed,'Bath':NBath,'Sqft':NSqft,'Lot_Size':NLot_Size,'Type':NType,'Price':NPrice,'Address':NAddress},ignore_index=True)

In [103]:
Point2_data.tail()

,Beds,Bath,Sqft,Lot_Size,Type,Price,Address
2096,4,1,2432,NaN,Single,179000,1837 Sandy Point Road
2097,NaN,NaN,NaN,.099,Residential,234900,15100 Cabot Trail Highway
2098,2,2,1082,.258,Residential,220000,800 Old Sambro Road
2099,3,NaN,2125,>4.6,Residential,259000,470 Patrice Road
2100,4,4,3109,NaN,Single,329000,288 South Foord Street


In [105]:
for column in Point2_data:
    print(str(column) + ' column has '+ str(Point2_data[column].isnull().sum()) + ' NaNs')

Beds column has 38 NaNs
Bath column has 36 NaNs
Sqft column has 36 NaNs
Lot_Size column has 1829 NaNs
Type column has 0 NaNs
Price column has 0 NaNs
Address column has 0 NaNs


In [104]:
Point2_data.to_csv('Point2_data_NS_house.csv',index=False)

### Now let's collect housing information in PEI

In [2]:
for i in range(1,8):
    url = 'https://www.point2homes.com/CA/Real-Estate-Maps.html?location=Prince+Edward+Island&PropertyType=House&search_mode=location&page=' + str(i) + '&SelectedView=map_view&LocationGeoId=205408&viewport=-65.28516031571866%2C45.3536041179178%2C-61.115848792281156%2C47.548272840428936&zoom=8&location_changed=&ajax=1'
    
print(url)

https://www.point2homes.com/CA/Real-Estate-Maps.html?location=Prince+Edward+Island&PropertyType=House&search_mode=location&page=7&SelectedView=map_view&LocationGeoId=205408&viewport=-65.28516031571866%2C45.3536041179178%2C-61.115848792281156%2C47.548272840428936&zoom=8&location_changed=&ajax=1


In [5]:
data = {}
soups = {}
tables_PE = []
# Create a df to store info
Point2_data = pd.DataFrame(columns=['Beds','Bath','Sqft','Lot_Size','Type','Price','Address'])
for i in range(1,8):
    url = 'https://www.point2homes.com/CA/Real-Estate-Maps.html?location=Prince+Edward+Island&PropertyType=House&search_mode=location&page=' + str(i) + '&SelectedView=map_view&LocationGeoId=205408&viewport=-65.28516031571866%2C45.3536041179178%2C-61.115848792281156%2C47.548272840428936&zoom=8&location_changed=&ajax=1'
    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')
    tables_PE.append(soup.find_all("div", class_="item-details"))
    
    # extract information from each table
    for table in (tables_PE[(i-1)]):
        if (len(str(table.find_all("li", class_='ic-beds')).split())<3):
            Nbed = float("NaN")
        else:
            Nbed = int(''.join(filter(str.isdigit, str(table.find_all("li", class_='ic-beds')))))
        
        if (len(str(table.find_all("li", class_='ic-baths')).split())<3):
            NBath = float("NaN")
        else:
            NBath = int(''.join(filter(str.isdigit, str(table.find_all("li", class_='ic-baths')))))
        
        if (len(str(table.find_all("li", class_='ic-sqft')).split())<3):
            NSqft = float("NaN")
        else:
            NSqft = int(''.join(filter(str.isdigit, str(table.find_all("li", class_='ic-sqft')))))
    
        if (len(str(table.find_all("li", class_='ic-lotsize')).split())<4):
            NLot_Size = float("NaN")
        else:
            NLot_Size = str(table.find_all("li", class_='ic-lotsize')).split()[4][-4:]
        
        if (len(str(table.find_all("li", class_='ic-proptype')).split())<3):
            NType = float("NaN")
        else:
            NType = str(table.find_all("li", class_='ic-proptype')).split()[3]


        NPrice = int(table.find("div", class_='price')['data-price'].replace('$','').replace(',','').replace('CAD',''))
        NAddress = table.find("div", class_='item-address')['data-address']
    # append data into the table 
        Point2_data = Point2_data.append({'Beds':Nbed,'Bath':NBath,'Sqft':NSqft,'Lot_Size':NLot_Size,'Type':NType,'Price':NPrice,'Address':NAddress},ignore_index=True)

In [6]:
Point2_data.tail()

,Beds,Bath,Sqft,Lot_Size,Type,Price,Address
601,4,3,2144,NaN,Residential,489000,45 Hillside Meadows
602,3,3,1625,ng>2,Residential,599900,251 New Seabury Drive
603,3,2,1800,>0.2,Residential,219900,2 Monaghan Street
604,4,2,1881,g>47,Farm,549000,1752 Rte 225
605,NaN,NaN,NaN,0.84,Residential,49900,Lot 42 Sunset Crescent


In [7]:
for column in Point2_data:
    print(str(column) + ' column has '+ str(Point2_data[column].isnull().sum()) + ' NaNs')

Beds column has 21 NaNs
Bath column has 23 NaNs
Sqft column has 37 NaNs
Lot_Size column has 247 NaNs
Type column has 0 NaNs
Price column has 0 NaNs
Address column has 0 NaNs


In [8]:
Point2_data.to_csv('Point2_data_PE_house.csv',index=False)

### Let's collect NB

In [9]:
for i in range(1,25):
    url = 'https://www.point2homes.com/CA/Real-Estate-Maps.html?location=New+Brunswick&PropertyType=House&search_mode=location&page=' + str(i) + '&SelectedView=map_view&LocationGeoId=205405&viewport=-70.5033065889374%2C44.07796464156973%2C-62.164683542062406%2C48.47860930093096&zoom=7&location_changed=&ajax=1'
    
print(url)

https://www.point2homes.com/CA/Real-Estate-Maps.html?location=New+Brunswick&PropertyType=House&search_mode=location&page=24&SelectedView=map_view&LocationGeoId=205405&viewport=-70.5033065889374%2C44.07796464156973%2C-62.164683542062406%2C48.47860930093096&zoom=7&location_changed=&ajax=1


In [122]:
data = {}
soups = {}
tables_NB = []
# Create a df to store info
Point2_data = pd.DataFrame(columns=['Beds','Bath','Sqft','Lot_Size','Type','Price','Address'])
for i in range(1,25):
    url = 'https://www.point2homes.com/CA/Real-Estate-Maps.html?location=New+Brunswick&PropertyType=House&search_mode=location&page=' + str(i) + '&SelectedView=map_view&LocationGeoId=205405&viewport=-70.5033065889374%2C44.07796464156973%2C-62.164683542062406%2C48.47860930093096&zoom=7&location_changed=&ajax=1'
    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')
    tables_NB.append(soup.find_all("div", class_="item-details"))
    
    # extract information from each table
    for table in (tables_NB[(i-1)]):
        if (len(str(table.find_all("li", class_='ic-beds')).split())<3):
            Nbed = float("NaN")
        else:
            Nbed = int(''.join(filter(str.isdigit, str(table.find_all("li", class_='ic-beds')))))
        
        if (len(str(table.find_all("li", class_='ic-baths')).split())<3):
            NBath = float("NaN")
        else:
            NBath = int(''.join(filter(str.isdigit, str(table.find_all("li", class_='ic-baths')))))
        
        if (len(str(table.find_all("li", class_='ic-sqft')).split())<3):
            NSqft = float("NaN")
        else:
            NSqft = int(''.join(filter(str.isdigit, str(table.find_all("li", class_='ic-sqft')))))
    
        if (len(str(table.find_all("li", class_='ic-lotsize')).split())<4):
            NLot_Size = float("NaN")
        else:
            NLot_Size = str(table.find_all("li", class_='ic-lotsize')).split()[4][-4:]
        
        if (len(str(table.find_all("li", class_='ic-proptype')).split())<3):
            NType = float("NaN")
        else:
            NType = str(table.find_all("li", class_='ic-proptype')).split()[3]


        NPrice = float(table.find("div", class_='price')['data-price'].replace('$','').replace(',','').replace('CAD',''))
        NAddress = table.find("div", class_='item-address')['data-address']
    # append data into the table 
        Point2_data = Point2_data.append({'Beds':Nbed,'Bath':NBath,'Sqft':NSqft,'Lot_Size':NLot_Size,'Type':NType,'Price':NPrice,'Address':NAddress},ignore_index=True)

In [123]:
Point2_data.tail()

,Beds,Bath,Sqft,Lot_Size,Type,Price,Address
2367,3.0,3.0,2350.0,NaN,Single,229000.0,30 Queen Street
2368,2.0,2.0,1200.0,0.23,Residential,159900.0,121 Sanford
2369,3.0,3.0,1760.0,.658,Residential,349900.0,736 Amirault ST
2370,2.0,1.0,NaN,NaN,Single,184000.0,397 Shore Road
2371,4.0,5.0,NaN,NaN,Single,599900.0,561 King George


In [124]:
for column in Point2_data:
    print(str(column) + ' column has '+ str(Point2_data[column].isnull().sum()) + ' NaNs')

Beds column has 120 NaNs
Bath column has 115 NaNs
Sqft column has 917 NaNs
Lot_Size column has 1122 NaNs
Type column has 0 NaNs
Price column has 0 NaNs
Address column has 0 NaNs


In [125]:
Point2_data.to_csv('Point2_data_NB_house.csv',index=False)

### Let's collect NL

In [10]:
for i in range(1,26):
    url = 'https://www.point2homes.com/CA/Real-Estate-Maps.html?location=Newfoundland+and+Labrador&PropertyType=House&search_mode=location&page=' + str(i) + '&SelectedView=map_view&LocationGeoId=205409&viewport=-70.5033065889374%2C44.07796464156973%2C-62.164683542062406%2C48.47860930093096&zoom=7&location_changed=true&ajax=1'
    
print(url)

https://www.point2homes.com/CA/Real-Estate-Maps.html?location=Newfoundland+and+Labrador&PropertyType=House&search_mode=location&page=25&SelectedView=map_view&LocationGeoId=205409&viewport=-70.5033065889374%2C44.07796464156973%2C-62.164683542062406%2C48.47860930093096&zoom=7&location_changed=true&ajax=1


In [127]:
data = {}
soups = {}
tables_NL = []
# Create a df to store info
Point2_data = pd.DataFrame(columns=['Beds','Bath','Sqft','Lot_Size','Type','Price','Address'])
for i in range(1,26):
    url = 'https://www.point2homes.com/CA/Real-Estate-Maps.html?location=Newfoundland+and+Labrador&PropertyType=House&search_mode=location&page=' + str(i) + '&SelectedView=map_view&LocationGeoId=205409&viewport=-70.5033065889374%2C44.07796464156973%2C-62.164683542062406%2C48.47860930093096&zoom=7&location_changed=true&ajax=1'
    data = requests.get(url).text
    soup = BeautifulSoup(data, 'html.parser')
    tables_NL.append(soup.find_all("div", class_="item-details"))
    
    # extract information from each table
    for table in (tables_NL[(i-1)]):
        if (len(str(table.find_all("li", class_='ic-beds')).split())<3):
            Nbed = float("NaN")
        else:
            Nbed = int(''.join(filter(str.isdigit, str(table.find_all("li", class_='ic-beds')))))
        
        if (len(str(table.find_all("li", class_='ic-baths')).split())<3):
            NBath = float("NaN")
        else:
            NBath = int(''.join(filter(str.isdigit, str(table.find_all("li", class_='ic-baths')))))
        
        if (len(str(table.find_all("li", class_='ic-sqft')).split())<3):
            NSqft = float("NaN")
        else:
            NSqft = int(''.join(filter(str.isdigit, str(table.find_all("li", class_='ic-sqft')))))
    
        if (len(str(table.find_all("li", class_='ic-lotsize')).split())<4):
            NLot_Size = float("NaN")
        else:
            NLot_Size = str(table.find_all("li", class_='ic-lotsize')).split()[4][-4:]
        
        if (len(str(table.find_all("li", class_='ic-proptype')).split())<3):
            NType = float("NaN")
        else:
            NType = str(table.find_all("li", class_='ic-proptype')).split()[3]


        NPrice = float(table.find("div", class_='price')['data-price'].replace('$','').replace(',','').replace('CAD',''))
        NAddress = table.find("div", class_='item-address')['data-address']
    # append data into the table 
        Point2_data = Point2_data.append({'Beds':Nbed,'Bath':NBath,'Sqft':NSqft,'Lot_Size':NLot_Size,'Type':NType,'Price':NPrice,'Address':NAddress},ignore_index=True)

In [128]:
Point2_data.tail()

,Beds,Bath,Sqft,Lot_Size,Type,Price,Address
2462,2,1,800,NaN,Residential,69900.0,9 Drakes Hill
2463,3,1,2056,NaN,Residential,149900.0,27 Gladstone Road
2464,3,1,1000,NaN,Single,60000.0,13 Merasheen Crescent
2465,3,1,2480,NaN,Residential,299900.0,40 Mint Cove Pond Road
2466,4,2,1904,NaN,Residential,199500.0,5 Frecker Place Extension


In [129]:
for column in Point2_data:
    print(str(column) + ' column has '+ str(Point2_data[column].isnull().sum()) + ' NaNs')

Beds column has 23 NaNs
Bath column has 23 NaNs
Sqft column has 10 NaNs
Lot_Size column has 2403 NaNs
Type column has 0 NaNs
Price column has 0 NaNs
Address column has 0 NaNs


In [130]:
Point2_data.to_csv('Point2_data_NL_house.csv',index=False)